In [16]:
from sklearn.metrics import mean_squared_error
import numpy as np
from tqdm import trange

In [29]:
class SGD:

    def __init__(self, sparse_matrix, K, lr, beta, n_epochs):
        """
        Arguments
        - sparse_matrix : user-item rating matrix
        - K (int)       : number of latent dimensions
        - lr (float) : learning rate
        - beta (float)  : regularization parameter
        - n_epochs (int) : Num of Iteration
        """
        # convert ndArray
        self.sparse_matrix = sparse_matrix.fillna(0).to_numpy()
        self.item_n, self.user_n = sparse_matrix.shape
        self.K = K
        self.lr = lr
        self.beta = beta
        self.n_epochs = n_epochs

    def train(self):
        # Initialize user and item latent feature matrice
        self.I = np.random.normal(scale=1./self.K, size=(self.item_n, self.K)) # scale = std
        self.U = np.random.normal(scale=1./self.K, size=(self.user_n, self.K))

        # Init biases
        self.item_bias = np.zeros(self.item_n)
        self.user_bias = np.zeros(self.user_n)
        self.total_mean = np.mean(self.sparse_matrix[np.where(self.sparse_matrix != 0)])

        # Create training Samples
        idx, jdx = self.sparse_matrix.nonzero()
        samples = list(zip(idx, jdx))

        training_log = []
        progress = trange(self.n_epochs, desc="train-rmse: nan")
        for idx in progress:
            np.random.shuffle(samples)

            for i, u in samples:
                # get error
                y = self.sparse_matrix[i, u]
                pred = self.predict(i, u)
                error = y - pred
                # update bias
                self.item_bias[i] += self.lr * (error - self.beta * self.item_bias[i])
                self.user_bias[u] += self.lr * (error - self.beta * self.user_bias[u])
                # update latent factors
                I_i = self.I[i,:][:]
                self.I[i, :] += self.lr * (error * self.U[u,:] - self.beta * self.I[i,:])
                self.U[u, :] += self.lr * (error * I_i - self.beta * self.U[u,:])

            rmse = self.evaluate()
            progress.set_description("train-rmse: %0.6f" % rmse)
            progress.refresh()
            training_log.append((idx, rmse))

        self.pred_matrix =  self.get_pred_matrix()

    def predict(self, i, u):
        """
        :param i: item index
        :param u: user index
        :return: predicted rating
        """
        return (
            self.total_mean
            + self.item_bias[i]
            + self.user_bias[u]
            + self.U[u,:].dot(self.I[i,:].T)
        )

    def get_pred_matrix(self):
        return (
            self.total_mean
            + self.item_bias[:,np.newaxis]
            + self.user_bias[np.newaxis:,]
            + self.I.dot(self.U.T)
        )

    def evaluate(self):
        idx, jdx = self.sparse_matrix.nonzero()
        pred_matrix = self.get_pred_matrix()
        ys, preds = [], []
        for i, j in zip(idx, jdx):
            ys.append(self.sparse_matrix[i, j])
            preds.append(pred_matrix[i, j])

        error = mean_squared_error(ys, preds)
        return np.sqrt(error)

    def test_evaluate(self, test_set):
        pred_matrix = self.get_pred_matrix()
        ys, preds = [], []
        for i, j, rating in test_set:
            ys.append(rating)
            preds.append(pred_matrix[i, j])

        error = mean_squared_error(ys, preds)
        return np.sqrt(error)

In [42]:
import pandas as pd

data = {
    'user1': [4, 3, np.nan, np.nan, 5, np.nan],
    'user2': [5, np.nan, 4, np.nan, np.nan, np.nan],
    'user3': [np.nan, np.nan, np.nan, 3, 4, np.nan],
    'user4': [1, 0, 0, np.nan, np.nan, 2],
}

movies = ['Movie A', 'Movie B', 'Movie C', 'Movie D', 'Movie E', 'Movie F']

movie_ratings = pd.DataFrame(data, index=movies)

In [43]:
movie_ratings

,user1,user2,user3,user4
Movie A,4.0,5.0,NaN,1.0
Movie B,3.0,NaN,NaN,0.0
Movie C,NaN,4.0,NaN,0.0
Movie D,NaN,NaN,3.0,NaN
Movie E,5.0,NaN,4.0,NaN
Movie F,NaN,NaN,NaN,2.0


In [44]:
# Create an instance of SGD with parameters
K = 3
lr = 0.01
beta = 0.02
n_epochs = 100

sgd_model = SGD(movie_ratings, K, lr, beta, n_epochs)

# Train the model
sgd_model.train()

# Evaluate the model on the training set
train_rmse = sgd_model.evaluate()
print("Root Mean Squared Error (RMSE) on training set:", train_rmse)

train-rmse: 0.391231: 100%|██████████| 100/100 [00:00<00:00, 1090.51it/s]

Root Mean Squared Error (RMSE) on training set: 0.3912305616780757


In [45]:
sgd_model.get_pred_matrix()

array([[3.87565366, 4.28032121, 3.33615821, 1.82322775],
       [3.09595719, 4.11915249, 3.35027517, 1.55116766],
       [3.57901389, 4.29619781, 3.26078626, 1.83467287],
       [3.98789476, 4.30744715, 3.06705145, 1.93692947],
       [4.92927549, 4.89975331, 3.98219644, 2.40962555],
       [3.79620312, 4.00078873, 3.27833015, 1.75567862]])